## Analog discovery using Simba

Modern metabolomics relies on tandem mass spectrometry (MS/MS) to identify the structures of small molecules in complex biological samples. While exact spectral matching against large reference libraries can confidently identify known compounds, discovering analogs, molecules with similar but not identical structures, remains a significant challenge. Analog search is crucial for applications ranging from drug lead optimization to biomarker discovery, where small structural changes can produce different biological effects.

**SIMBA** (Spectral Identification of Molecule Bio-Analogues) is a deep-learning framework designed to bridge this gap. Rather than rely only on heuristic scores like modified cosine, SIMBA’s twin‐transformer architecture learns to predict two chemically interpretable metrics directly from MS/MS spectra:

1. **Substructure Edit Distance (SED):** the minimal number of bond edits (additions/removals) needed to transform one molecule into another via their Maximum Common Substructure (MCS).  
2. **Maximum Common Edge Subgraph (MCES) Distance:** the count of bond differences outside the shared subgraph, offering a complementary view of structural similarity.

This notebook walks you through a typical analog-discovery workflow using SIMBA:

1. **Compute Spectral Embeddings**  
   We begin by computing the MS/MS embeddings generated by SIMBA’s transformer encoders for both query and reference spectra.

2. **Perform Analog Search**  
   We compute pairwise MCES and SED predictions between query spectra (e.g., from the CASMI benchmark) and a reference library ( MassSpecGym) and rank candidates by structural distance.

3. **Evaluate examples**  
   We evaluate some examples of the matches found by SIMBA.

By the end of this notebook, you’ll see how SIMBA’s learned similarity metrics enable more sensitive and interpretable analog retrieval.


In [ ]:
%load_ext autoreload
%autoreload 2

## Libraries

In [ ]:
import simba
from simba.config import Config
from simba.core.models.simba_model import Simba
from simba.core.data.preprocessing_simba import PreprocessingSimba
from simba.utils.plotting_mces import Plotting
from simba.analog_discovery.simba_analog_discovery import AnalogDiscovery
import numpy as np
import spectrum_utils.plot as sup
from rdkit import Chem
from simba.core.data.ground_truth import GroundTruth
import matplotlib.pyplot as plt
import spectrum_utils.plot as sup

## Defining parameters

Define the default configuration variables

In [ ]:
config=Config()
config.USE_LEARNABLE_MULTITASK=True
config.USE_FINGERPRINT=False
FILTER_SPECTRA_BY_PRECURSOR_MZ= False ## if you want to filter your search for only matches close to your query spectra
FILTER_SPECTRA_TOLERANCE= 0.5

Location of model saved, reference spectra in mgf file (MASSSPECGYM), an query spectra (CASMI)

In [ ]:
model_location = '/Users/sebas/projects/data/best_model_20250422_only_massspecgym.ckpt'
reference_file = '/Users/sebas/projects/data/MassSpecGym.mgf'
#casmi_file= '/Users/sebas/projects/data/processed_massformer/casmi2022_spec_df.pkl' 
casmi_file= '/Users/sebas/Downloads/MCR_only_POS_iimn_gnps.mgf' 

## Load spectra

Let's load the reference spectra and query spectra. This code already carries out a preprocessing of the files obtaining only protonized adducts and spectra with at least more than 6 peaks.

In [ ]:
all_spectrums_reference=PreprocessingSimba.load_spectra(reference_file, config, use_gnps_format=False)

In [ ]:
print(f'Number of spectra loaded from reference: {len(all_spectrums_reference)}')

In [ ]:
all_spectrums_query=PreprocessingSimba.load_spectra(casmi_file, config, use_gnps_format=False)

In [ ]:
all_spectrums_query[0].params

In [ ]:
print(f'Number of spectra loaded from query: {len(all_spectrums_query)}')

##  Let's check some spectra visually

In [ ]:
all_spectrums_query[2].params

In [ ]:
sup.spectrum(all_spectrums_query[2])

## Initialize model

Here we load a simba model based on path specified in 'model_location'. The device to be used is set to 'cpu' unless you have access to a configures GPU. The argument cache_embeddings, allows to reuse embeddings already computed to accelerate future library searchs.

In [ ]:
simba_model= Simba(model_location, config=config, device='cpu', cache_embeddings=True)

## Predictions

Based on the simba model created let's predict the substructure edit distance (sim_ed) and MCES distance (sim_mces)

In [ ]:
sim_ed, sim_mces= simba_model.predict(all_spectrums_query, all_spectrums_reference, )

In [ ]:
sim_mces

The predictions of substructure edit distance are discretized between 0 and 5, being 5 having five or more modifications and 0 having zero modifications. Let's take 10,000 random predictions and check the distribution of the results. Higher substructure edit distances are more common since related molecules are scarse normally.

In [ ]:
flat = sim_ed.ravel()                            # view as 1-D array of length M*N
idx = np.random.choice(flat.size, size=10000, replace=False)
samples_ed = flat[idx]
plt.hist(samples_ed, bins=20)
plt.grid()
plt.xlabel('substructure edit distance')
plt.ylabel('Frequency')

The predictions of MCES distance are constrained to 0 to 40 edges. Let's take 10,000 random predictions and check the distribution of the results. Higher MCES distances are more common since related molecules are scarse normally.

In [ ]:
flat = sim_mces.ravel()                            # view as 1-D array of length M*N
idx = np.random.choice(flat.size, size=10000, replace=False)
samples_mces = flat[idx]
plt.hist(samples_mces, bins=20)
plt.grid()
plt.xlabel('MCES distance')
plt.ylabel('Frequency')

## Reranking

Based on the predictions of MCES and Edit distance we can rerank the results. Lower MCES distance and lower edit distances are higher in the rank. The MCES distance is used as primary metric to rank the predictions given its finer granularity. If 2 predictions have the same MCES distance, the one with the lower substructure edit distance is ranked higher.

In [ ]:
ranking= AnalogDiscovery.compute_ranking(sim_mces, sim_ed)

The rank is scaled to 0-1 (normalized to the number of comparisons with the reference library), where 1 means the highest ranking and 0 the lowest ranking.

In [ ]:
ranking.shape

In [ ]:
flat = ranking.ravel()                            # view as 1-D array of length M*N
idx = np.random.choice(flat.size, size=10000, replace=False)
samples_mces = flat[idx]
plt.hist(samples_mces, bins=20)
plt.grid()
plt.xlabel('ranking score')
plt.ylabel('Frequency')

## What is the matched spectra in the reference library for each query spectra?

If we want to find this answer, we have to first select the query spectra we are interested. We can define a variable 'target_index' which indicates the position of the spectrum in the spectra loaded. From there, we can select the 10 highest SIMBA scores and filtering the match with the lowest MCES distance

In [ ]:
target_index=10 # target spectrum to analyze
N=20 #number of matches to be analyzed

In [ ]:
spectra_query= all_spectrums_query[target_index]

Let's checkwhat are the predictions of the model for this specific query spectra. These are the ranking scores generated for the query spectra selected:

In [ ]:
ranking[target_index]

In [ ]:
ranking[target_index].shape

## Let's take a window for the precursor mass 

In [ ]:
#best_matches= np.argsort(ranking[target_index]*close_precursor_mz_boolean)[-100:]
if FILTER_SPECTRA_BY_PRECURSOR_MZ:
    close_precursor_mz = [abs(s.precursor_mz - spectra_query.precursor_mz)/spectra_query.precursor_mz for s in all_spectrums_reference]
    close_precursor_mz_boolean= [1 if c<FILTER_SPECTRA_TOLERANCE else 0 for c in close_precursor_mz]
    best_matches= np.argsort(ranking[target_index]*close_precursor_mz_boolean)[-N:]
else:
    best_matches= np.argsort(ranking[target_index])[-N:]

In [ ]:
best_matches

In [ ]:
spectra_matches = [all_spectrums_reference[ind] for ind in best_matches]

Let's compute the MCES distances for the matches found and select the best

Let's select the lowest MCES distance

In [ ]:
best_match_index = best_matches[0]

In [ ]:
best_match_index

## What is the precursor mass of the matches found?

In [ ]:
spectra_query.params

In [ ]:
spectra_query.precursor_mz

In [ ]:
[s.params['precursor_mz'] for s in spectra_matches]

In [ ]:
spectra_match= spectra_matches[0]

This is the index in the reference spectra with the best match:

In [ ]:
Chem.MolFromSmiles(spectra_match.params['smiles'])

Now take a look at what the model predicts in terms of substructure edit distance and MCES distance for this specfic comparison:

In [ ]:
print(f'The MCES distance predicted is: {np.round(sim_mces[target_index, best_match_index])}')

In [ ]:
print(f'The substructure edit distance predicted is: {sim_ed[target_index, best_match_index]}')

## How it looks like the spectra that is found as match?

In [ ]:
sup.mirror(spectra_query, spectra_match)